In [2]:
from cdapython import Q, columns, unique_terms, query


### Top level files for a single subject

In [3]:
import json
q1 = Q('id = "TCGA-13-1504"')
r = q1.run()
print(r)


Getting results from database

Total execution time: 1409 ms

QueryID: cf96b9a2-da6d-4b61-9c52-7947e2226f9a
Query: SELECT all_v2.* FROM gdc-bq-sample.integration.all_v2 AS all_v2 WHERE (all_v2.id = 'TCGA-13-1504')
Offset: 0
Count: 1
Total Row Count: 1
More pages: False



In [4]:
from collections import Counter
fileCounter = Counter()
for f in r[0]['File']:
    fileCounter[f['data_category']] +=1
print(fileCounter)

Counter({'Imaging': 226, 'Peptide Spectral Matches': 55, 'Raw Mass Spectra': 28, 'Processed Mass Spectra': 27, 'Simple Nucleotide Variation': 16, 'Biospecimen': 12, 'Clinical': 8, 'Copy Number Variation': 7, 'Sequencing Reads': 3, 'Transcriptome Profiling': 2, 'DNA Methylation': 1, 'Proteome Profiling': 1})


Looking at the top level Sequencing Reads files

In [6]:
for f in r[0]['File']:
    if f['data_category'] == "Sequencing Reads":
        print(json.dumps(f,indent=3))

{
   "id": "8b7859ed-395a-449d-8e4b-7d385d0ffa43",
   "identifier": [
      {
         "system": "GDC",
         "value": "8b7859ed-395a-449d-8e4b-7d385d0ffa43"
      }
   ],
   "label": "TCGA-13-1504-01A-01R-1565-13_mirna_gdc_realn.bam",
   "data_category": "Sequencing Reads",
   "data_type": "Aligned Reads",
   "file_format": "BAM",
   "associated_project": "TCGA-OV",
   "drs_uri": "drs://dg.4DFC:8b7859ed-395a-449d-8e4b-7d385d0ffa43",
   "byte_size": "617903851",
   "checksum": "42b51bf61122b716cd02a628da9b7d89"
}
{
   "id": "698994fe-b22a-4723-b9f0-23551fd3bb0e",
   "identifier": [
      {
         "system": "GDC",
         "value": "698994fe-b22a-4723-b9f0-23551fd3bb0e"
      }
   ],
   "label": "C239.TCGA-13-1504-01A-01W.5_gdc_realn.bam",
   "data_category": "Sequencing Reads",
   "data_type": "Aligned Reads",
   "file_format": "BAM",
   "associated_project": "TCGA-OV",
   "drs_uri": "drs://dg.4DFC:698994fe-b22a-4723-b9f0-23551fd3bb0e",
   "byte_size": "32100267850",
   "checksum"

### Examine the files within ResearchSubjects and Specimens

In [7]:
from collections import Counter
fileCounts = Counter()
fileReferences = {}
data_cat = "Sequencing Reads"
for su in r[0]['ResearchSubject']:
    for f in su['File']:
        if f['data_category'] == data_cat:        
            fileCounts[f['label']] +=1
            fileReferences[f['label']] = {'referenced_by':[{"type":"subject","id":su['id']}]}
    for sp in su['Specimen']:
         for f in sp['File']:
            if f['data_category'] == data_cat:
                fileCounts[f['label']] +=1
                specDetails = {k: sp[k] for k in ('id', 'source_material_type', 'specimen_type', 'derived_from_specimen')}
                specDetails['type']= 'specimen'
                fileReferences[f['label']]['referenced_by'].append(specDetails)
print(fileCounts)

Counter({'TCGA-13-1504-01A-01R-1565-13_mirna_gdc_realn.bam': 7, 'C239.TCGA-13-1504-01A-01W.5_gdc_realn.bam': 7, 'C239.TCGA-13-1504-10A-01W.6_gdc_realn.bam': 5})


In [8]:
import pandas as pd
from IPython.display import display, HTML

for f, v in fileReferences.items():
    refs = v['referenced_by']
    df = pd.DataFrame(refs)
    print("File:{}".format(f))
    print("File is included in the following objects")
    display(df)
    print('_'*80)
    print()

File:TCGA-13-1504-01A-01R-1565-13_mirna_gdc_realn.bam
File is included in the following objects


/Users/forei/imfenvs/cda2/lib/python3.9/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,type,id,source_material_type,specimen_type,derived_from_specimen
0,subject,cd49126a-ec15-43fa-9e43-3f7460d43f2b,NaN,NaN,NaN
1,specimen,78fbf568-1a01-4725-8994-bf084b12e316,Primary Tumor,sample,initial specimen
2,specimen,5d4e2a9b-4960-4c07-9aab-f9a11c8d11df,Primary Tumor,portion,78fbf568-1a01-4725-8994-bf084b12e316
3,specimen,2224d488-a25a-4d13-bf62-6b8518cc67d4,Primary Tumor,slide,5d4e2a9b-4960-4c07-9aab-f9a11c8d11df
4,specimen,8b60e9fd-1119-4e92-a606-881b1daecbf6,Primary Tumor,slide,5d4e2a9b-4960-4c07-9aab-f9a11c8d11df
5,specimen,5f2d28ea-0750-4aa0-863c-9db59d93bfc5,Primary Tumor,analyte,5d4e2a9b-4960-4c07-9aab-f9a11c8d11df
6,specimen,4eb038e4-6e3c-40c5-abde-c446204e6cb6,Primary Tumor,aliquot,5f2d28ea-0750-4aa0-863c-9db59d93bfc5


________________________________________________________________________________

File:C239.TCGA-13-1504-01A-01W.5_gdc_realn.bam
File is included in the following objects


,type,id,source_material_type,specimen_type,derived_from_specimen
0,subject,cd49126a-ec15-43fa-9e43-3f7460d43f2b,NaN,NaN,NaN
1,specimen,78fbf568-1a01-4725-8994-bf084b12e316,Primary Tumor,sample,initial specimen
2,specimen,5d4e2a9b-4960-4c07-9aab-f9a11c8d11df,Primary Tumor,portion,78fbf568-1a01-4725-8994-bf084b12e316
3,specimen,2224d488-a25a-4d13-bf62-6b8518cc67d4,Primary Tumor,slide,5d4e2a9b-4960-4c07-9aab-f9a11c8d11df
4,specimen,8b60e9fd-1119-4e92-a606-881b1daecbf6,Primary Tumor,slide,5d4e2a9b-4960-4c07-9aab-f9a11c8d11df
5,specimen,b05701a3-1ab8-46a1-a32e-cf6e58de519e,Primary Tumor,analyte,5d4e2a9b-4960-4c07-9aab-f9a11c8d11df
6,specimen,15c62cbb-78de-42a2-acd8-46599cd61986,Primary Tumor,aliquot,b05701a3-1ab8-46a1-a32e-cf6e58de519e


________________________________________________________________________________

File:C239.TCGA-13-1504-10A-01W.6_gdc_realn.bam
File is included in the following objects


,type,id,source_material_type,specimen_type,derived_from_specimen
0,subject,cd49126a-ec15-43fa-9e43-3f7460d43f2b,NaN,NaN,NaN
1,specimen,2d46dc86-53ca-4eba-989f-239951507ad7,Blood Derived Normal,sample,initial specimen
2,specimen,7cbe22ca-5491-48fd-835e-44466927ffdd,Blood Derived Normal,portion,2d46dc86-53ca-4eba-989f-239951507ad7
3,specimen,a5061825-bae8-4294-8366-67a260fa6f48,Blood Derived Normal,analyte,7cbe22ca-5491-48fd-835e-44466927ffdd
4,specimen,4a43a38c-892c-4478-b94f-318dbe282269,Blood Derived Normal,aliquot,a5061825-bae8-4294-8366-67a260fa6f48


________________________________________________________________________________



One particularly odd thing is that the files are referenced from the Slide. The actual physical relationship between these objects is as follows

```
Portion 5d4e2a9b-4960-4c07-9aab-f9a11c8d11df
--->Analyte 5f2d28ea-0750-4aa0-863c-9db59d93bfc5
------>Aliquot 4eb038e4-6e3c-40c5-abde-c446204e6cb6	
--------->File TCGA-13-1504-01A-01R-1565-13_mirna_gdc_realn.bam
--->Slide 2224d488-a25a-4d13-bf62-6b8518cc67d4
--->Slide 8b60e9fd-1119-4e92-a606-881b1daecbf6
```

It is misleading to represent the file as associated with the slides.

### Use the GDC API to see what entities are associated with the files

The following queries the Genomic Data Commons to see what entity each of these three files is associated with.


In [25]:
import requests

gdc_api_file_endpt = 'https://api.gdc.cancer.gov/files/'
headers = {'Content-Type': 'application/json'}
fields = "file_name,associated_entities.entity_id,associated_entities.entity_type"

for f in fileReferences.keys():
    body = {
        "filters":{
                "op":"=",
                "content":{
                    "field":"file_name",
                    "value":f
                }

        },
        "fields":fields,
        "format":"json",
    }
    resp = requests.post(gdc_api_file_endpt, data=json.dumps(body), headers=headers)
    print(resp.json()['data']['hits'][0])
    print()



{'id': '8b7859ed-395a-449d-8e4b-7d385d0ffa43', 'associated_entities': [{'entity_type': 'aliquot', 'entity_id': '4eb038e4-6e3c-40c5-abde-c446204e6cb6'}], 'file_name': 'TCGA-13-1504-01A-01R-1565-13_mirna_gdc_realn.bam'}

{'id': '698994fe-b22a-4723-b9f0-23551fd3bb0e', 'associated_entities': [{'entity_type': 'aliquot', 'entity_id': '15c62cbb-78de-42a2-acd8-46599cd61986'}], 'file_name': 'C239.TCGA-13-1504-01A-01W.5_gdc_realn.bam'}

{'id': '07a9e595-023d-47d5-97fa-3fcc62f5d0c6', 'associated_entities': [{'entity_type': 'aliquot', 'entity_id': '4a43a38c-892c-4478-b94f-318dbe282269'}], 'file_name': 'C239.TCGA-13-1504-10A-01W.6_gdc_realn.bam'}



The GDC associates these files only with the aliquot entity.